In [1]:
import numpy as np
import os, pickle, sys, json, random, math 
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, add, concatenate, TimeDistributed, Bidirectional
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import Dense, Input, Flatten, Merge, Dropout, concatenate, Concatenate, merge
from sklearn.metrics import accuracy_score
import nltk
import tensorflow as tf

/home/gauravroy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('/home/gauravroy/Desktop/intent/embedding_matrix.pickle', 'rb') as fp:
    embedding_matrix = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/intent/train_sequence.pickle', 'rb') as fp:
    train_sequence = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/intent/test_sequence.pickle', 'rb') as fp:
    test_sequence = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/intent/intent_slot_word_dict.pickle', 'rb') as fp:
    intent_slot_word_dict = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/intent/train_idata.pickle', 'rb') as fp:
    itrain = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/intent/test_idata.pickle', 'rb') as fp:
    itest = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/intent/y_train_intent.pickle', 'rb') as fp:
    y_train_intent = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/intent/y_test_intent.pickle', 'rb') as fp:
    y_test_intent = pickle.load(fp)
fp.close()

intent_dict, words_dict = intent_slot_word_dict

max_len = 47

In [3]:
z_train_intent = list()
z_test_intent = list()

for out in y_train_intent:
    for i in range(len(out)):
        if(out[i] == 1):
            z_train_intent.append(i)
        
for out in y_test_intent:
    for i in range(len(out)):
        if(out[i] == 1):
            z_test_intent.append(i)

In [4]:
z_test_intent

[4,
 5,
 9,
 6,
 7,
 8,
 9,
 4,
 6,
 4,
 6,
 4,
 4,
 4,
 5,
 5,
 6,
 6,
 6,
 0,
 9,
 2,
 9,
 9,
 1,
 1,
 8,
 6,
 5,
 2,
 5,
 4,
 4,
 9,
 4,
 4,
 4,
 4,
 6,
 5,
 4,
 4,
 9,
 1,
 6,
 1,
 2,
 5,
 1,
 1,
 4,
 5,
 4,
 6,
 6,
 4,
 4,
 4,
 9,
 6,
 4,
 5,
 4,
 9,
 6,
 6,
 9,
 9,
 5,
 4,
 4,
 6,
 4,
 9,
 5,
 5,
 9,
 4,
 5,
 6,
 6,
 4,
 1,
 5,
 6,
 5,
 9,
 4,
 5,
 4,
 4,
 5,
 4,
 4,
 4,
 6,
 4,
 4,
 5,
 4,
 4,
 4,
 4,
 5,
 6,
 6,
 6,
 0,
 4,
 6,
 9,
 6,
 9,
 9,
 5,
 4,
 6,
 4,
 4,
 4,
 6,
 4,
 6,
 6,
 9,
 6,
 9,
 4,
 4,
 2,
 4,
 6,
 9,
 5,
 5,
 5,
 6,
 4,
 4,
 5,
 9,
 5,
 2,
 6,
 9,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 6,
 1,
 4,
 4,
 2,
 2,
 4,
 5,
 5,
 9,
 6,
 9,
 5,
 6,
 9,
 6,
 6,
 8,
 9,
 6,
 9,
 6,
 9,
 8,
 9,
 4,
 6,
 6,
 6,
 6,
 9,
 6,
 6,
 5,
 9,
 6,
 6,
 4,
 5,
 6,
 8,
 5,
 9,
 6,
 9,
 1,
 1,
 5,
 4,
 4,
 4,
 4,
 6,
 4,
 4,
 6,
 6,
 2,
 6,
 6,
 4,
 4,
 4,
 5,
 2,
 5,
 8,
 5,
 2,
 5,
 4,
 4,
 4,
 4,
 9,
 5,
 2,
 4,
 9,
 4,
 4,
 4,
 2,
 4,
 4,
 4,
 6,
 6,
 6,
 6,
 4,
 6,
 6,
 9,
 5,
 4,
 1,
 9,


In [5]:
def accu(list1, list2):
    if(len(list1) != len(list2)):
        print("Size of a the lists not equal")
        return
    count = 0.0
    for i in range(len(list1)):
        if(list1[i] == list2[i]):
            count = count + 1
            
    return count/len(list1)  

# 3 CNN With Different Kernel Sizes

In [6]:
cnn_embedding_layer = Embedding(len(embedding_matrix), len(embedding_matrix[0]), weights=[embedding_matrix], input_length=max_len, trainable=False)

sequence_input = Input(shape=(max_len,), dtype='int32')

cnn_embedded_sequences = cnn_embedding_layer(sequence_input)

conv0 = Conv1D(filters=100, kernel_size=2, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool0 = MaxPooling1D(2)(conv0)
flatten0 = Flatten()(pool0)

conv1 = Conv1D(filters=100, kernel_size=3, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool1 = MaxPooling1D(2)(conv1)
flatten1 = Flatten()(pool1)
                                                # identity and check
conv2 = Conv1D(filters=100, kernel_size=4, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool2 = MaxPooling1D(2)(conv2)
flatten2 = Flatten()(pool2)

# out = Merge(mode='concat')([flatten0,flatten1,flatten2])
out = concatenate([flatten0,flatten1,flatten2])

graph = Model(inputs=sequence_input, outputs=out)

cnn_model = Sequential()
cnn_model.add(graph)
cnn_model.add(Dropout(0.10))
cnn_model.add(Dense(output_dim=20, activation='relu'))
cnn_model.add(Dense(units=len(intent_dict), activation='softmax',kernel_initializer='he_normal'))


/home/gauravroy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=20)`


In [7]:
graph.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 47)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 47, 600)      514800      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 46, 100)      120100      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 45, 100)      180100      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

In [8]:
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','categorical_accuracy'])
#              optimizer='sgd' optimizer='adam',metrics=[metrics.mae, metrics.categorical_accuracy])

saveBestModel = ModelCheckpoint(filepath="cnn_weights.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

cnn_model.fit(train_sequence, np.array(y_train_intent),validation_data=(test_sequence,np.array(y_test_intent)), epochs=10, batch_size=50, verbose=2, callbacks=[saveBestModel])
cnn_model.load_weights("cnn_weights.hdf5")

Train on 5355 samples, validate on 1336 samples
Epoch 1/10
 - 13s - loss: 1.4002 - acc: 0.5785 - categorical_accuracy: 0.5785 - val_loss: 1.0910 - val_acc: 0.7036 - val_categorical_accuracy: 0.7036

Epoch 00001: val_acc improved from -inf to 0.70359, saving model to cnn_weights.hdf5
Epoch 2/10
 - 11s - loss: 0.8700 - acc: 0.7374 - categorical_accuracy: 0.7374 - val_loss: 0.9832 - val_acc: 0.7163 - val_categorical_accuracy: 0.7163

Epoch 00002: val_acc improved from 0.70359 to 0.71632, saving model to cnn_weights.hdf5
Epoch 3/10
 - 11s - loss: 0.7493 - acc: 0.7677 - categorical_accuracy: 0.7677 - val_loss: 0.9218 - val_acc: 0.7433 - val_categorical_accuracy: 0.7433

Epoch 00003: val_acc improved from 0.71632 to 0.74326, saving model to cnn_weights.hdf5
Epoch 4/10
 - 12s - loss: 0.6602 - acc: 0.7955 - categorical_accuracy: 0.7955 - val_loss: 0.9165 - val_acc: 0.7320 - val_categorical_accuracy: 0.7320

Epoch 00004: val_acc did not improve from 0.74326
Epoch 5/10
 - 11s - loss: 0.5866 - ac

In [9]:
cnn_pred_classes_train = cnn_model.predict_classes(train_sequence)
cnn_pred_classes_test = cnn_model.predict_classes(test_sequence)

In [11]:
np.shape(cnn_pred_classes_train)

(5355,)

In [12]:
np.shape(train_sequence)

(5355, 47)

In [10]:
print ("Train Accuracy: " + str(accu(z_train_intent,cnn_pred_classes_train)*100) + "%")
print ("Test Accuracy: " + str(accu(z_test_intent,cnn_pred_classes_test)*100) + "%")                

Train Accuracy: 88.12324929971989%
Test Accuracy: 75.0%


# 3 CNN with LSTM and Same Kernel_Size

In [33]:
cnn_embedding_layer = Embedding(len(embedding_matrix), len(embedding_matrix[0]), weights=[embedding_matrix], input_length=max_len, trainable=False)

sequence_input = Input(shape=(max_len,), dtype='int32')

cnn_embedded_sequences = cnn_embedding_layer(sequence_input)

conv0 = Conv1D(filters=100, kernel_size=3, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool0 = MaxPooling1D(2)(conv0)
flatten0 = Flatten()(pool0)

conv1 = Conv1D(filters=100, kernel_size=3, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool1 = MaxPooling1D(2)(conv1)
flatten1 = Flatten()(pool1)
                                                # identity and check
conv2 = Conv1D(filters=100, kernel_size=3, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool2 = MaxPooling1D(2)(conv2)
#pool2  = pool2.get_shape()
flatten2 = Flatten()(pool2)

# out = Merge(mode='concat')([flatten0,flatten1,flatten2])
out = concatenate([pool0,pool1,pool2])

lstm = LSTM(200,return_sequences=False)(out)

out = Dense(units=200, activation='relu', kernel_initializer='he_normal')(lstm)

out = Dense(units=len(intent_dict), activation='softmax', kernel_initializer='he_normal')(out)

In [34]:
graph = Model(inputs=sequence_input, outputs=out)
cnn_model = Sequential()
cnn_model.add(graph)
#cnn_model.add(get_shape(47*2))
#cnn_model.add(get_shape(None,22,100))
cnn_model.add(Dropout(0.10))
cnn_model.add(Dense(output_dim=20, activation='relu'))
cnn_model.add(Dense(units=len(intent_dict), activation='softmax',kernel_initializer='he_normal'))

/home/gauravroy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=20)`
  import sys


In [35]:
graph.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 47)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 47, 600)      514800      input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 45, 100)      180100      embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 45, 100)      180100      embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_9 (

In [36]:
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','categorical_accuracy'])
#              optimizer='sgd' optimizer='adam',metrics=[metrics.mae, metrics.categorical_accuracy])
saveBestModel = ModelCheckpoint(filepath="cnn_weights.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
cnn_model.fit(np.array(train_sequence), np.array(y_train_intent),validation_data=(np.array(test_sequence),np.array(y_test_intent)), epochs=10, batch_size=50, verbose=2, callbacks=[saveBestModel])
#cnn_model.load_weights("cnn_weights.hdf5")

Train on 5355 samples, validate on 1336 samples
Epoch 1/10
 - 19s - loss: 2.1992 - acc: 0.3302 - categorical_accuracy: 0.3302 - val_loss: 1.9662 - val_acc: 0.4499 - val_categorical_accuracy: 0.4499

Epoch 00001: val_acc improved from -inf to 0.44985, saving model to cnn_weights.hdf5
Epoch 2/10
 - 21s - loss: 1.8064 - acc: 0.3864 - categorical_accuracy: 0.3864 - val_loss: 1.6951 - val_acc: 0.4626 - val_categorical_accuracy: 0.4626

Epoch 00002: val_acc improved from 0.44985 to 0.46257, saving model to cnn_weights.hdf5
Epoch 3/10
 - 22s - loss: 1.6801 - acc: 0.3798 - categorical_accuracy: 0.3798 - val_loss: 1.6388 - val_acc: 0.4626 - val_categorical_accuracy: 0.4626

Epoch 00003: val_acc improved from 0.46257 to 0.46257, saving model to cnn_weights.hdf5
Epoch 4/10
 - 22s - loss: 1.5860 - acc: 0.4069 - categorical_accuracy: 0.4069 - val_loss: 1.5193 - val_acc: 0.5277 - val_categorical_accuracy: 0.5277

Epoch 00004: val_acc improved from 0.46257 to 0.52769, saving model to cnn_weights.hdf5

In [75]:
cnn_model.load_weights("cnn_weights.hdf5")

In [37]:
cnn_pred_classes_train = cnn_model.predict_classes(train_sequence)
cnn_pred_classes_test = cnn_model.predict_classes(test_sequence)

In [38]:
print ("Train Accuracy: " + str(accu(z_train_intent,cnn_pred_classes_train)*100) + "%")
print ("Test Accuracy: " + str(accu(z_test_intent,cnn_pred_classes_test)*100) + "%")                

Train Accuracy: 54.957983193277315%
Test Accuracy: 58.38323353293413%


# 3 CNN with Bidirectional LSTM and Same Kernel_size

In [16]:
cnn_embedding_layer = Embedding(len(embedding_matrix), len(embedding_matrix[0]), weights=[embedding_matrix], input_length=max_len, trainable=False)

sequence_input = Input(shape=(max_len,), dtype='int32')

cnn_embedded_sequences = cnn_embedding_layer(sequence_input)

conv0 = Conv1D(filters=100, kernel_size=3, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool0 = MaxPooling1D(2)(conv0)
flatten0 = Flatten()(pool0)

conv1 = Conv1D(filters=100, kernel_size=3, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool1 = MaxPooling1D(2)(conv1)
flatten1 = Flatten()(pool1)
                                                # identity and check
conv2 = Conv1D(filters=100, kernel_size=3, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool2 = MaxPooling1D(2)(conv2)
#pool2  = pool2.get_shape()
flatten2 = Flatten()(pool2)

# out = Merge(mode='concat')([flatten0,flatten1,flatten2])
out = concatenate([pool0,pool1,pool2])

lstm = Bidirectional(LSTM(200,return_sequences=False))(out)
#lstm = tf.reshape(lstm,[400,1], name=None)
out = Dense(units=200, activation='relu', kernel_initializer='he_normal')(lstm)

out = Dense(units=len(intent_dict), activation='softmax', kernel_initializer='he_normal')(out)

In [17]:
np.shape(out)

TensorShape([Dimension(None), Dimension(12)])

In [18]:
np.shape(lstm)

TensorShape([Dimension(None), Dimension(400)])

In [19]:
np.shape(sequence_input)

TensorShape([Dimension(None), Dimension(47)])

In [20]:
np.shape(out)

TensorShape([Dimension(None), Dimension(12)])

In [21]:
graph = Model(inputs=sequence_input, outputs=out)

In [22]:
cnn_model = Sequential()
cnn_model.add(graph)
#cnn_model.add(get_shape(47*2))
#cnn_model.add(get_shape(None,22,100))
cnn_model.add(Dropout(0.10))
cnn_model.add(Dense(output_dim=20, activation='relu'))
cnn_model.add(Dense(units=len(intent_dict), activation='softmax',kernel_initializer='he_normal'))

/home/gauravroy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=20)`
  


In [23]:
graph.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 47)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 47, 600)      514800      input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 45, 100)      180100      embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 45, 100)      180100      embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (

In [24]:
#y_train_intent = np.array(y_train_intent)

In [25]:
#y_train_intent = y_train_intent.reshape(5355,12,1)

In [26]:
#np.shape(y_train_intent)

In [27]:
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','categorical_accuracy'])
#              optimizer='sgd' optimizer='adam',metrics=[metrics.mae, metrics.categorical_accuracy])

In [28]:
saveBestModel = ModelCheckpoint(filepath="cnn_weights.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

In [29]:
cnn_model.fit(np.array(train_sequence), np.array(y_train_intent),validation_data=(np.array(test_sequence),np.array(y_test_intent)), epochs=10, batch_size=50, verbose=2, callbacks=[saveBestModel])
#cnn_model.load_weights("cnn_weights.hdf5")

Train on 5355 samples, validate on 1336 samples
Epoch 1/10
 - 31s - loss: 2.2215 - acc: 0.3175 - categorical_accuracy: 0.3175 - val_loss: 1.9331 - val_acc: 0.5344 - val_categorical_accuracy: 0.5344

Epoch 00001: val_acc improved from -inf to 0.53443, saving model to cnn_weights.hdf5
Epoch 2/10
 - 33s - loss: 1.7638 - acc: 0.4726 - categorical_accuracy: 0.4726 - val_loss: 1.5768 - val_acc: 0.5696 - val_categorical_accuracy: 0.5696

Epoch 00002: val_acc improved from 0.53443 to 0.56961, saving model to cnn_weights.hdf5
Epoch 3/10
 - 32s - loss: 1.5441 - acc: 0.4792 - categorical_accuracy: 0.4792 - val_loss: 1.4703 - val_acc: 0.5704 - val_categorical_accuracy: 0.5704

Epoch 00003: val_acc improved from 0.56961 to 0.57036, saving model to cnn_weights.hdf5
Epoch 4/10
 - 32s - loss: 1.4172 - acc: 0.5117 - categorical_accuracy: 0.5117 - val_loss: 1.3927 - val_acc: 0.5928 - val_categorical_accuracy: 0.5928

Epoch 00004: val_acc improved from 0.57036 to 0.59281, saving model to cnn_weights.hdf5

In [30]:
cnn_model.load_weights("cnn_weights.hdf5")

In [31]:
cnn_pred_classes_train = cnn_model.predict_classes(train_sequence)
cnn_pred_classes_test = cnn_model.predict_classes(test_sequence)

In [32]:
print ("Train Accuracy: " + str(accu(z_train_intent,cnn_pred_classes_train)*100) + "%")
print ("Test Accuracy: " + str(accu(z_test_intent,cnn_pred_classes_test)*100) + "%")                

Train Accuracy: 75.31279178338002%
Test Accuracy: 71.63173652694611%


# 3 CNN With LSTM And Different Kernel size

In [11]:
cnn_embedding_layer = Embedding(len(embedding_matrix), len(embedding_matrix[0]), weights=[embedding_matrix], input_length=max_len, trainable=False)

sequence_input = Input(shape=(max_len,), dtype='int32')

cnn_embedded_sequences = cnn_embedding_layer(sequence_input)

conv0 = Conv1D(filters=100, kernel_size=2, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool0 = MaxPooling1D(2)(conv0)
flatten0 = Flatten()(pool0)

conv1 = Conv1D(filters=100, kernel_size=3, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool1 = MaxPooling1D(2)(conv1)
flatten1 = Flatten()(pool1)
                                                # identity and check
conv2 = Conv1D(filters=100, kernel_size=4, activation='relu', padding='valid',kernel_initializer='he_normal')(cnn_embedded_sequences)
pool2 = MaxPooling1D(2)(conv2)
#pool2  = pool2.get_shape()
flatten2 = Flatten()(pool2)

# out = Merge(mode='concat')([flatten0,flatten1,flatten2])
out = merge([pool0, pool1, pool2], mode='concat', concat_axis=1)
#out = concatenate([pool0,pool1,pool2])

lstm = LSTM(200,return_sequences=False)(out)
#lstm = tf.reshape(lstm,[400,1], name=None)
out = Dense(units=200, activation='relu', kernel_initializer='he_normal')(lstm)

out = Dense(units=len(intent_dict), activation='softmax', kernel_initializer='he_normal')(out)

/home/gauravroy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/gauravroy/.local/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [13]:
graph = Model(inputs=sequence_input, outputs=out)
cnn_model = Sequential()
cnn_model.add(graph)
#cnn_model.add(get_shape(47*2))
#cnn_model.add(get_shape(None,22,100))
cnn_model.add(Dropout(0.10))
cnn_model.add(Dense(output_dim=20, activation='relu'))
cnn_model.add(Dense(units=len(intent_dict), activation='softmax',kernel_initializer='he_normal'))

/home/gauravroy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=20)`
  import sys


In [71]:
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','categorical_accuracy'])
#              optimizer='sgd' optimizer='adam',metrics=[metrics.mae, metrics.categorical_accuracy])

saveBestModel = ModelCheckpoint(filepath="cnn_weights.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

cnn_model.fit(train_sequence, np.array(y_train_intent),validation_data=(test_sequence,np.array(y_test_intent)), epochs=10, batch_size=50, verbose=2, callbacks=[saveBestModel])
cnn_model.load_weights("cnn_weights.hdf5")

Train on 5355 samples, validate on 1336 samples
Epoch 1/10
 - 31s - loss: 2.2552 - acc: 0.1908 - categorical_accuracy: 0.1908 - val_loss: 2.0478 - val_acc: 0.2994 - val_categorical_accuracy: 0.2994

Epoch 00001: val_acc improved from -inf to 0.29940, saving model to cnn_weights.hdf5
Epoch 2/10
 - 33s - loss: 2.0555 - acc: 0.2050 - categorical_accuracy: 0.2050 - val_loss: 1.9656 - val_acc: 0.1766 - val_categorical_accuracy: 0.1766

Epoch 00002: val_acc did not improve from 0.29940
Epoch 3/10
 - 33s - loss: 2.0053 - acc: 0.2209 - categorical_accuracy: 0.2209 - val_loss: 1.9632 - val_acc: 0.1766 - val_categorical_accuracy: 0.1766

Epoch 00003: val_acc did not improve from 0.29940
Epoch 4/10
 - 34s - loss: 1.9903 - acc: 0.2217 - categorical_accuracy: 0.2217 - val_loss: 1.9598 - val_acc: 0.1766 - val_categorical_accuracy: 0.1766

Epoch 00004: val_acc did not improve from 0.29940
Epoch 5/10
 - 33s - loss: 1.9811 - acc: 0.2217 - categorical_accuracy: 0.2217 - val_loss: 1.9623 - val_acc: 0.176

In [72]:
cnn_pred_classes_train = cnn_model.predict_classes(train_sequence)
cnn_pred_classes_test = cnn_model.predict_classes(test_sequence)

In [73]:
print ("Train Accuracy: " + str(accu(z_train_intent,cnn_pred_classes_train)*100) + "%")
print ("Test Accuracy: " + str(accu(z_test_intent,cnn_pred_classes_test)*100) + "%")                

Train Accuracy: 20.11204481792717%
Test Accuracy: 29.94011976047904%
